# Week 6 Lab Assignment

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [25]:
from sqlalchemy import create_engine


#### My first step is to load in the dataset that I finished up with last week.

In [26]:
host = r'127.0.0.1'
db = r'MSDS610'
user = r'postgres'
pw = r'MacMVP2025$' 
port = r'5432'

In [27]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [28]:
table_name = r'real_estate_clean5'
schema = r'cleaned'

real_clean = pd.read_sql_table(table_name, db_conn, schema)

In [29]:
real_clean.head()

,List Year,Non Use Code,Assessed Value,Sale Amount,Sales Ratio,Overpriced
0,2022,0.0,139020.0,232000.0,0.599224,1
1,2020,0.0,253000.0,430000.0,0.588372,1
2,2021,0.0,329730.0,805000.0,0.409602,1
3,2020,0.0,130400.0,179900.0,0.724847,0
4,2020,0.0,619290.0,890000.0,0.695831,0


In [30]:
real_clean = real_clean.sample(250)

## Analytical Question

#### Can we predict a foreclosure based on the sales ratio, list year, and sale amount?

#### I plan to use Non Use Code as the feature we're looking to predict and the sales ratio, list year, and sale amount to try to make that prediction.

#### First, I need to get setup to split the data. I'm creating a reference for the columns in my dataset, setting X and y values, and identifying a prediction feature.

In [31]:
cols = real_clean.columns


prediction_col = 'Non Use Code'
feature_cols = [c for c in cols if c != prediction_col]
X = real_clean[feature_cols]
y = real_clean[prediction_col]

In [32]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=42, test_size=0.3, stratify=y)

#### Looks like we are all set here but I'm going to test these out to make sure they're displaying what I want them to.

In [33]:
X_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 4788 to 496206
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   List Year       75 non-null     int64  
 1   Assessed Value  75 non-null     float64
 2   Sale Amount     75 non-null     float64
 3   Sales Ratio     75 non-null     float64
 4   Overpriced      75 non-null     int64  
dtypes: float64(3), int64(2)
memory usage: 3.5 KB


In [34]:
X_temp.head()

,List Year,Assessed Value,Sale Amount,Sales Ratio,Overpriced
4788,2022,1976590.0,3995000.0,0.494766,1
145872,2022,307650.0,400000.0,0.769125,0
648385,2010,240500.0,190745.0,1.260846,0
639900,2010,102250.0,120000.0,0.852083,0
1018133,2019,197640.0,290000.0,0.681517,0


In [35]:
y_temp.head()

4788       0.0
145872     0.0
648385     0.0
639900     0.0
1018133    0.0
Name: Non Use Code, dtype: float64

#### Now I need to train my validation set, I will do that below.

In [36]:
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, random_state=42, test_size=0.5, stratify=y_temp)

#### And again, testing things out to make sure they look right.

In [37]:
print(X_test.shape)
X_test.head()

(37, 5)


,List Year,Assessed Value,Sale Amount,Sales Ratio,Overpriced
406368,2004,118720.0,355000.0,0.334423,1
613861,2009,190200.0,276000.0,0.689130,0
774275,2014,189630.0,268041.0,0.707466,0
716496,2012,125720.0,32500.0,3.868308,0
969088,2018,179410.0,247000.0,0.726356,0


In [38]:
print(X_val.shape)
X_val.head()

(38, 5)


,List Year,Assessed Value,Sale Amount,Sales Ratio,Overpriced
781748,2013,185400.0,274500.0,0.675410,0
710051,2012,158310.0,225000.0,0.703600,0
63145,2020,109220.0,140000.0,0.780143,0
639900,2010,102250.0,120000.0,0.852083,0
127392,2022,442120.0,795000.0,0.556126,1


#### The next step now is to create my Random Forest Classifier model.

In [39]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [40]:
predict_x_test = model.predict(X_test)

In [41]:
accuracy_score(y_test,predict_x_test)

0.9459459459459459

#### I happy with my accuracy score now, 0.95 is better than I could have hoped for. I believe this is my optimal model.

#### My last steps for this notebook are to save my model using joblib and uploading my data to PostgreSQL so that I can access it in another notebook.

In [42]:
model_name = "real_clean_rf.joblib"
joblib.dump(model, model_name)

['real_clean_rf.joblib']

In [43]:
host = r'127.0.0.1'
db = r'MSDS610'
user = r'postgres'
pw = r'MacMVP2025$' 
port = r'5432'
schema = r'analytic' 

In [44]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))


In [45]:
real_clean1 = r'real_clean'

In [47]:
real_clean.to_sql(real_clean1, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')


250